In [ ]:
import matplotlib.pyplot as plt
from data import get_radiomics_dataset
from torchvision import transforms
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics

import lime
import lime.lime_tabular

In [ ]:
train_data, train_labels, val_data, val_labels, test_data, test_labels = get_radiomics_dataset()

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
# get null values
null_values = train_data.isnull().sum().sum()
print('Null:', null_values)
# get nan values
nan_values = np.isnan(train_data.values).sum()
print('Nan:', nan_values)

In [ ]:
X_train = np.array(train_data.values)
X_val = np.array(val_data.values)
X_test = np.array(test_data.values)

X = np.concatenate((X_train, X_val))
Y = np.concatenate((train_labels, val_labels))

feature_list = list(train_data.columns)
X_train.shape

### First attempt at running a model

In [ ]:
# Instantiate random forest and train on new features
rf_clf = RandomForestClassifier(n_estimators= 1000, random_state=1, n_jobs=-1, verbose = 1)
rf_clf.fit(X_train, train_labels)

In [ ]:
y_val_pred=rf_clf.predict(X_val)

from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(val_labels, y_val_pred))

## Hyperparameter Search for Random Forest

We need to find a better performing hyperparameter combination through randomized search. Since the dataset is very small, searching based only one one validation set can easily cause overfitting on that set, so we combine the train and validation sets and carry out 5 fold cross validation on the combined dataset instead.

### Hyperparameter Space:

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

### Searching for Hyperparameters:

In [ ]:
X = np.concatenate((X_train, X_val))
Y = np.concatenate((train_labels, val_labels))

In [ ]:
# Use the random grid to search for best hyperparameters
rf = RandomForestClassifier()

# Use the list to create PredefinedSplit
#pds = PredefinedSplit(test_fold = split_index)

# Use PredefinedSplit in RandomizedSearchCV
clf = RandomizedSearchCV(estimator = rf, cv=5, param_distributions=random_grid,
                          n_iter = 300, verbose=4, random_state=1, n_jobs = -1 )

# Fit with all data
clf.fit(X, Y)

```
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=300,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 15, 26, 36, 47, 57, 68,
                                                      78, 89, 100],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=1, verbose=4)
```

In [ ]:
clf.best_params_

best_params
```
{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 47,
 'bootstrap': False}
```

## Final Baseline Result

In [ ]:
try:
    best_model = clf.best_estimator_
except NameError:
    best_model = RandomForestClassifier(n_estimators=800, min_samples_split =10, min_samples_leaf=4, max_features='auto', max_depth=47, bootstrap=False)
    best_model.fit(X, Y)

y_test_pred=best_model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_labels, y_test_pred))
print("F1 Score:",metrics.f1_score(test_labels, y_test_pred))

cm = confusion_matrix(test_labels, y_test_pred, labels=best_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=best_model.classes_).plot()
plt.show()

```
Accuracy: 0.7857142857142857
F1 Score: 0.8500000000000001
```

![](Plots/CM_RandomForestBaseline.png)

## Interpretability of Random Forest using LIME

### Creating model explainer

In [ ]:
predict_func = lambda x: best_model.predict_proba(x).astype(float)
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,feature_names = train_data.columns,class_names=['No Tumor','Tumor'],kernel_width=5)

In [ ]:
sample = val_data.loc[[4]].values[0]
exp = explainer.explain_instance(data_row=val_data.loc[4], predict_fn=best_model.predict_proba ,num_features=10)
exp.show_in_notebook(show_table=True)

In [ ]:
sample2 = val_data.values[5]
exp = explainer.explain_instance(sample2, predict_func,num_features=5, num_samples=500)
exp.show_in_notebook(show_all=False)

In [ ]:
sample3 = val_data.values[10]
exp = explainer.explain_instance(sample3, predict_func,num_features=15, num_samples=500)
exp.show_in_notebook(show_all=False)

## SHAP

In [ ]:
import shap 

# Create Tree Explainer object that can calculate shap values
explainer = shap.TreeExplainer(best_model)

index = 10
sample = test_data.iloc[index]

shap_values = explainer.shap_values(sample)
print(f"Prediction: {best_model.predict(sample.values.reshape(1,-1))[0]}, True Label: {test_labels[index]}")
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], sample)

In [ ]:
shap_values = explainer.shap_values(test_data)
shap.summary_plot(shap_values, test_data)

In [ ]:
# Feature importance based on random forest MDI
importances = best_model.feature_importances_
indices = np.argpartition(importances, -10)[-10:]
features = train_data.columns
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

# Feature importance based on random forest
result = permutation_importance(
    best_model, X_test, test_labels, n_repeats=10, random_state=42, n_jobs=-1
)

sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx[-10:]].T, vert=False, labels=train_data.columns[sorted_idx[-10:]]
)
ax.set_title("Permutation Importances (test set)")
ax.set_xlabel('Relative Importance')
fig.tight_layout()
plt.show()

## Feature Selection --- doesnt work well, may be discarded

We observe whether selecting a subset of dataset features improves model performance.

In [ ]:
from sklearn.feature_selection import  f_classif 
k=50

select = SelectKBest(score_func=f_classif, k=k)
X_train_kbest = select.fit_transform(X_train,train_labels)
X_val_kbest = select.transform(X_val)
X_test_kbest = select.transform(X_test)


In [ ]:

# Use the random grid to search for best hyperparameters
rf_kbest = RandomForestClassifier()


X = np.concatenate((X_train_kbest, X_val_kbest))
Y = np.concatenate((train_labels, val_labels))

# Use the list to create PredefinedSplit
#pds = PredefinedSplit(test_fold = split_index)

# Use PredefinedSplit in RandomizedSearchCV
clf_kbest = RandomizedSearchCV(estimator = rf_kbest, cv=5, param_distributions=random_grid,
                          n_iter = 100, verbose=4, random_state=1, n_jobs = -1 )

# Fit with all data
clf_kbest.fit(X, Y)

In [ ]:
clf_kbest.best_params_

In [ ]:
best_model = clf_kbest.best_estimator_
y_test_pred=best_model.predict(X_test_kbest)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_labels, y_test_pred))
print("F1 Score:",metrics.f1_score(test_labels, y_test_pred))

cm = confusion_matrix(test_labels, y_test_pred, labels=best_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=best_model.classes_).plot()
plt.show()